In [6]:
# !pip install torchvision 
# !pip install pytorch

In [5]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [50]:
# !pip install matplotlib

In [53]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [4]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# from ipywidgets import FloatProgress

In [5]:
#import dataset

train_data = datasets.FashionMNIST(root= 'data', 
                                  train= True,
                                  download =True,
                                  transform = ToTensor())

test_data = datasets.FashionMNIST(root= 'data', 
                                  train= False,
                                  download =True,
                                  transform = ToTensor())

86.6%

100.6%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



119.3%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [36]:
#Data Loader

batch_size = 64

#create dataloader
train_dataloader =DataLoader(train_data, batch_size=batch_size)
test_dataloader =DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("shape of the data N C H W ", X.shape)
    print("shape of the data", y.shape)
    break

shape of the data N C H W  torch.Size([64, 1, 28, 28])
shape of the data torch.Size([64])


In [56]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_dataloader), size=(1,)).item()
    img, label = train_dataloader[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

TypeError: 'DataLoader' object is not subscriptable

<Figure size 576x576 with 0 Axes>

## Creating MOdel

In [11]:
#check cuda is available or not
device = "cuda" if torch.cuda.is_available() else "cpu"
print("using Device", device)

using Device cpu


In [16]:
# define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten =nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),
                                              nn.ReLU(),
                                              nn.Linear(512, 512),
                                              nn.ReLU(),
                                              nn.Linear(512, 10))
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model  = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
#To train model we need optimizer and loss function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [37]:
#train model
def trian(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #compute the prediction error
        pred= model(X)
        loss =loss_fn(pred, y)
        
        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100==0:
            loss, current = loss.item(), batch *len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [40]:
#test model

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches =len(dataloader)
    model.eval()
    test_loss, correct=0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
            
    test_loss /=num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
# Now training time

epochs = 5

for i in range(epochs):
    print("epochs", i+1)
    trian(train_dataloader,model,loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")


epochs 1
loss: 1.920032  [    0/60000]
loss: 1.897195  [ 6400/60000]
loss: 1.780011  [12800/60000]
loss: 1.829267  [19200/60000]
loss: 1.703800  [25600/60000]
loss: 1.658326  [32000/60000]
loss: 1.688088  [38400/60000]
loss: 1.592140  [44800/60000]
loss: 1.618363  [51200/60000]
loss: 1.512211  [57600/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.522348 

epochs 2
loss: 1.582400  [    0/60000]
loss: 1.556580  [ 6400/60000]
loss: 1.404407  [12800/60000]
loss: 1.480355  [19200/60000]
loss: 1.345383  [25600/60000]
loss: 1.344698  [32000/60000]
loss: 1.369031  [38400/60000]
loss: 1.294800  [44800/60000]
loss: 1.332670  [51200/60000]
loss: 1.231376  [57600/60000]
Test Error: 
 Accuracy: 63.4%, Avg loss: 1.251387 

epochs 3
loss: 1.322897  [    0/60000]
loss: 1.314589  [ 6400/60000]
loss: 1.146277  [12800/60000]
loss: 1.253257  [19200/60000]
loss: 1.116517  [25600/60000]
loss: 1.143358  [32000/60000]
loss: 1.176162  [38400/60000]
loss: 1.112651  [44800/60000]
loss: 1.155324  [51200/60000]

In [43]:
#Save model
torch.save(model.state_dict(), "model.pth")
print("Save your model on ")

Save your model on 


In [44]:
#Load Model

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [45]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


                                               ___ Happy Ending with QuickStart___